In [1]:
##### add python path #####
import sys
import os
PATH = os.getcwd()
for dir_idx, dir_name in enumerate(PATH.split('/')):
    if 'optimal_control' in dir_name.lower():
        PATH = '/'.join(PATH.split('/')[:(dir_idx+1)])
        break
if not PATH in sys.path:
    sys.path.append(PATH)
###########################

from trajectory_planner import TrajectoryPlanner
from gait_scheduler import GaitScheduler
from env.quadruped.env import Env

from scipy.spatial.transform import Rotation
from copy import deepcopy
import numpy as np
import time

In [2]:
env = Env(enable_draw=False, base_fix=False)
num_leg = env.num_leg
time_step = env.time_step

gs_args = dict()
gs_args['num_leg'] = num_leg
gs_args['gait_period'] = 1.0
gs_args['is_visualize'] = False
gait_scheduler = GaitScheduler(gs_args)

tp_args = dict()
tp_args['time_step'] = 0.05
tp_args['time_horizon'] = 20
tp_args['max_accel'] = 0.1
tp_args['max_ang_accel'] = 0.1
trajectory_planner = TrajectoryPlanner(tp_args)

In [3]:
gait_list = np.zeros((num_leg, 2)) # FR, FL, RR, RL
gait_list[0,0] = 0.5 # offset
gait_list[0,1] = 0.5 # duration
gait_list[2,0] = 0.0 # offset
gait_list[2,1] = 0.5 # duration
gait_scheduler.gait_list = deepcopy(gait_list)
gait_scheduler.update()

vel_cmd = np.zeros(3)
vel_cmd[0] = 0.2
vel_cmd[1] = 0.0
ang_vel_cmd = np.zeros(3)
ang_vel_cmd[2] = 0.0
height_cmd = 0.35

In [4]:
env.reset()
init_state = np.concatenate([env.model.com_pos, env.model.base_rpy, env.model.base_vel, env.model.base_ang_vel])
target_trajectory_list = trajectory_planner.get_target_trajectory_list(vel_cmd, ang_vel_cmd, height_cmd, init_state)
contact_list, step_info_list = gait_scheduler.get_contact_list(trajectory_planner.time_step, trajectory_planner.time_horizon)

In [14]:
gait_scheduler.step(0.05)
contact_list, step_info_list = gait_scheduler.get_contact_list(trajectory_planner.time_step, trajectory_planner.time_horizon)
print(contact_list)
print(step_info_list)

[[0. 1. 1. 1.]
 [0. 1. 1. 1.]
 [0. 1. 1. 1.]
 [0. 1. 1. 1.]
 [0. 1. 1. 1.]
 [0. 1. 1. 1.]
 [0. 1. 1. 1.]
 [0. 1. 1. 1.]
 [0. 1. 1. 1.]
 [0. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
[[False, [[1.0, 0, 10]]], [False, []], [False, []], [False, []]]


In [5]:
curr_foot_pos_list = deepcopy(env.model.foot_pos_list)
print(curr_foot_pos_list)

[[ 0.1844826  -0.13916866 -0.02764445]
 [ 0.1844826   0.13916866 -0.02764445]
 [-0.1955174  -0.13916866 -0.02764445]
 [-0.1955174   0.13916866 -0.02764445]]


In [6]:
step_info_list

[[False, [[1.0, 10, 20]]], [False, []], [False, [[1.0, 0, 10]]], [False, []]]

In [7]:
contact_list

array([[1., 1., 0., 1.],
       [1., 1., 0., 1.],
       [1., 1., 0., 1.],
       [1., 1., 0., 1.],
       [1., 1., 0., 1.],
       [1., 1., 0., 1.],
       [1., 1., 0., 1.],
       [1., 1., 0., 1.],
       [1., 1., 0., 1.],
       [1., 1., 0., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.]])

In [17]:
time_horizon = trajectory_planner.time_horizon
target_foot_step_list = np.zeros((time_horizon, num_leg, 3))
curr_foot_pos_list = deepcopy(env.model.foot_pos_list)
pre_step_list = np.zeros((num_leg, 3))
for leg_idx in range(num_leg):
    step_info = step_info_list[leg_idx]
    if len(step_info[1]) == 0:
        target_foot_step_list[:,leg_idx, :] = curr_foot_pos_list[leg_idx]
        pre_step_list[leg_idx, :] = curr_foot_pos_list[leg_idx]
        continue

    is_swinging, temp_step_info_list = step_info

    pre_idx = 0
    pre_foot_pos = curr_foot_pos_list[leg_idx]
    for step_idx in range(len(temp_step_info_list)):
        stand_period, idx1, idx2 = temp_step_info_list[step_idx]
        target_foot_step_list[pre_idx:idx1, leg_idx, :] = pre_foot_pos
        pre_idx = idx2
        if step_idx == 0 and is_swinging:
            pre_foot_pos = pre_step_list[leg_idx]
        else:
            state = target_trajectory_list[idx2 - 1]
            pos_com = state[:3]
            rpy_base = state[3:6]
            vel_base = state[6:9]
            ang_vel_base = state[9:]

            r = Rotation.from_euler('z', rpy_base[2], degrees=False)
            pre_foot_pos = pos_com + np.matmul(r.as_matrix(), env.model.abduct_org_list[leg_idx])
            pre_foot_pos += stand_period*0.5*vel_base + 0.03*(vel_base - vel_cmd)
            pre_foot_pos += 0.5*np.sqrt(height_cmd/9.8)*np.cross(vel_base, ang_vel_cmd)
            pre_foot_pos[2] = 0.0

        if step_idx == 0:
            pre_step_list[leg_idx, :] = pre_foot_pos
    target_foot_step_list[pre_idx:time_horizon, leg_idx, :] = pre_foot_pos

print(target_foot_step_list)

[[[ 0.1844826  -0.13916866 -0.02764445]
  [ 0.1844826   0.13916866 -0.02764445]
  [ 0.          0.          0.        ]
  [-0.1955174   0.13916866 -0.02764445]]

 [[ 0.1844826  -0.13916866 -0.02764445]
  [ 0.1844826   0.13916866 -0.02764445]
  [ 0.          0.          0.        ]
  [-0.1955174   0.13916866 -0.02764445]]

 [[ 0.1844826  -0.13916866 -0.02764445]
  [ 0.1844826   0.13916866 -0.02764445]
  [ 0.          0.          0.        ]
  [-0.1955174   0.13916866 -0.02764445]]

 [[ 0.1844826  -0.13916866 -0.02764445]
  [ 0.1844826   0.13916866 -0.02764445]
  [ 0.          0.          0.        ]
  [-0.1955174   0.13916866 -0.02764445]]

 [[ 0.1844826  -0.13916866 -0.02764445]
  [ 0.1844826   0.13916866 -0.02764445]
  [ 0.          0.          0.        ]
  [-0.1955174   0.13916866 -0.02764445]]

 [[ 0.1844826  -0.13916866 -0.02764445]
  [ 0.1844826   0.13916866 -0.02764445]
  [ 0.          0.          0.        ]
  [-0.1955174   0.13916866 -0.02764445]]

 [[ 0.1844826  -0.13916866 -

In [18]:
pre_step_list

array([[ 0.2945    , -0.049     ,  0.        ],
       [ 0.1844826 ,  0.13916866, -0.02764445],
       [-0.15075   , -0.049     ,  0.        ],
       [-0.1955174 ,  0.13916866, -0.02764445]])